In [5]:

import os

os.chdir("..")
from langchain_community.chat_models import ChatHuggingFace
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

from apps.inners.use_cases.embeddings.bge_m3_embedding import BgeM3Embedding

from apps.inners.use_cases.document_converters.libre_office_document_converter import LibreOfficeDocumentConverter
from apps.inners.use_cases.document_converters.marker_document_converter import MarkerDocumentConverter
from apps.inners.use_cases.graphs.long_form_qa_graph import LongFormQaGraph
from apps.inners.use_cases.graphs.passage_search_graph import PassageSearchGraph
from apps.inners.use_cases.long_form_qas.process_long_form_qa import ProcessLongFormQa
from apps.inners.use_cases.managements.document_management import DocumentManagement
from apps.inners.use_cases.managements.file_document_management import FileDocumentManagement
from apps.inners.use_cases.managements.text_document_management import TextDocumentManagement
from apps.inners.use_cases.managements.web_document_management import WebDocumentManagement
from apps.inners.use_cases.passage_searches.process_passage_search import ProcessPassageSearch
from apps.outers.datastores.four_datastore import FourDatastore
from apps.outers.datastores.one_datastore import OneDatastore
from apps.outers.datastores.temp_datastore import TempDatastore
from apps.outers.datastores.three_datastore import ThreeDatastore
from apps.outers.datastores.two_datastore import TwoDatastore
from apps.outers.repositories.file_document_repository import FileDocumentRepository
from apps.outers.repositories.text_document_repository import TextDocumentRepository
from apps.outers.repositories.web_document_repository import WebDocumentRepository
from apps.outers.settings.one_embedding_setting import OneEmbeddingSetting
from apps.outers.settings.one_llm_setting import OneLlmSetting
from tests.seeders.all_seeder import AllSeeder

from apps.inners.models.dtos.contracts.requests.long_form_qas.input_setting_body import GeneratorSetting

from apps.inners.models.dtos.contracts.requests.long_form_qas.input_setting_body import \
    InputSettingBody as LongFormQaInputSettingBody
from apps.inners.models.dtos.contracts.requests.passage_searches.input_setting_body import \
    InputSettingBody as PassageSearchInputSettingBody, LlmSetting, \
    PreprocessorSetting, EmbedderSetting, RetrieverSetting, RerankerSetting
from apps.inners.models.dtos.contracts.requests.passage_searches.process_body import \
    ProcessBody as PassageSearchProcessBody
from apps.inners.models.dtos.contracts.requests.long_form_qas.process_body import ProcessBody as LongFormQaProcessBody

from langchain_anthropic import ChatAnthropic
from ragas.testset import TestsetGenerator, evolutions

from apps.inners.use_cases.graphs.preparation_graph import PreparationGraph
from apps.inners.use_cases.document_processor.category_document_processor import CategoryDocumentProcessor
from apps.inners.use_cases.document_processor.partition_document_processor import PartitionDocumentProcessor
from apps.inners.use_cases.document_processor.summary_document_processor import SummaryDocumentProcessor

from tools import cache_tool

import gc

from starlette.datastructures import State

import dotenv
from datasets import load_dataset
from dotenv import find_dotenv
from ragas import evaluate, metrics

from tests.containers.test_container import TestContainer
from apps.inners.models.dtos.graph_state import LongFormQaGraphState, PreparationGraphState


2024-07-24 12:43:47.305479: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-24 12:43:47.312786: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-24 12:43:47.321265: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-24 12:43:47.323763: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-24 12:43:47.331716: I tensorflow/core/platform/cpu_feature_guar

In [14]:
!pip show pytest

Name: pytest
Version: 8.2.1
Summary: pytest: simple powerful testing with Python
Home-page: 
Author: Holger Krekel, Bruno Oliveira, Ronny Pfannschmidt, Floris Bruynooghe, Brianna Laugher, Florian Bruhin, Others (See AUTHORS)
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: exceptiongroup, iniconfig, packaging, pluggy, tomli
Required-by: pytest-asyncio, pytest-cov, pytest-xdist


In [7]:
!python3 -m pytest --cov=./apps --cov-report=html ./tests -n 4

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
============================= test session starts ==============================
platform linux -- Python 3.10.12, pytest-8.3.1, pluggy-1.5.0
rootdir: /tmp/pycharm_project_305
plugins: cov-5.0.0, asyncio-0.23.8, xdist-3.6.1, anyio-4.4.0
asyncio: mode=strict
4 workers [43 items]   m
...........................................                              [100%]
=============================== warnings summary ===============================
../../usr/local/lib/python3.10/dist-packages/pydantic/_internal/_config.py:291: 10 warnings
  /usr/local/lib/python3.10/dist-packages/pydantic/_internal/_config.py:291: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
    warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)

../../usr/local/lib/pytho

In [4]:
# !pip show flagembedding
# !pip show langchain-anthropic
# !pip show pymilvus
# !pip show opencv-python
# !libreoffice --help
# !wkhtmltopdf
# !apt install strace
# !strace -e open,openat python3 -c "import tensorflow as tf" 2>&1 | grep "libnvinfer\|TF-TRT"
# !echo $(echo $(dirname $(dirname $(python3 -c "import nvidia.cudnn;print(nvidia.cudnn.__file__)")))/*/lib/ | sed -r 's/\s+/:/g')${LD_LIBRARY_PATH:+:${LD_LIBRARY_PATH}}
# !ls /TensorRT-8.6.1.6
# !python3 -m langchain_core.sys_info
!pip show onnxruntime-gpu

Name: onnxruntime-gpu
Version: 1.17.0
Summary: ONNX Runtime is a runtime accelerator for Machine Learning models
Home-page: https://onnxruntime.ai
Author: Microsoft Corporation
Author-email: onnxruntime@microsoft.com
License: MIT License
Location: /usr/local/lib/python3.10/dist-packages
Requires: coloredlogs, flatbuffers, numpy, packaging, protobuf, sympy
Required-by: 


In [1]:
from onnxruntime.capi import _pybind_state as C

print(f"Available ONNXRT providers: {C.get_available_providers()}")

Available ONNXRT providers: ['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'CPUExecutionProvider']


In [1]:
import tensorflow
from tensorflow.compiler.tf2tensorrt._pywrap_py_utils import get_linked_tensorrt_version, get_loaded_tensorrt_version

print(f"Linked TensorRT version {get_linked_tensorrt_version()}")
# print(f"Loaded TensorRT version {get_loaded_tensorrt_version()}")

tensorflow.config.list_physical_devices('GPU')

2024-07-24 10:20:02.649693: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-24 10:20:02.657863: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-24 10:20:02.667071: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-24 10:20:02.669620: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-24 10:20:02.676938: I tensorflow/core/platform/cpu_feature_guar

Linked TensorRT version (8, 6, 1)


I0000 00:00:1721816404.186395   19929 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1721816404.317047   19929 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1721816404.317087   19929 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [6]:
# Sanity check if TensorRT is working.
import numpy as np
from tensorflow import keras


def get_model():
    # Create a simple model.
    inputs = keras.Input(shape=(32,))
    outputs = keras.layers.Dense(1)(inputs)
    model = keras.Model(inputs, outputs)
    model.compile(optimizer="adam", loss="mean_squared_error")
    return model


model = get_model()

# Train the model.
test_input = np.random.random((128, 32))
test_target = np.random.random((128, 1))
model.fit(test_input, test_target)

# Calling `save('my_model')` creates a SavedModel folder `my_model`.
model.save("my_model.keras")

!aria2c https://raw.githubusercontent.com/tensorflow/tensorrt/master/tftrt/blog_posts/Leveraging%20TensorFlow-TensorRT%20integration%20for%20Low%20latency%20Inference/tf2_inference.py
!python3 tf2_inference.py --use_tftrt_model --precision fp16

In [4]:
import torch

torch.cuda.is_available()

In [2]:
dotenv.load_dotenv(find_dotenv())


In [10]:
test_container: TestContainer = TestContainer()

one_llm_setting: OneLlmSetting = test_container.applications.settings.one_llm()
one_embedding_setting: OneEmbeddingSetting = test_container.applications.settings.one_embedding()

one_datastore: OneDatastore = test_container.applications.datastores.one()
two_datastore: TwoDatastore = test_container.applications.datastores.two()
three_datastore: ThreeDatastore = test_container.applications.datastores.three()
four_datastore: FourDatastore = test_container.applications.datastores.four()
temp_datastore: TempDatastore = test_container.applications.datastores.temp()

file_document_repository: FileDocumentRepository = test_container.applications.repositories.file_document()
text_document_repository: TextDocumentRepository = test_container.applications.repositories.text_document()
web_document_repository: WebDocumentRepository = test_container.applications.repositories.web_document()

libre_office_document_converter: LibreOfficeDocumentConverter = test_container.applications.use_cases.document_converter.libre_office()
marker_document_converter: MarkerDocumentConverter = test_container.applications.use_cases.document_converter.marker()
document_management: DocumentManagement = test_container.applications.use_cases.managements.document()
file_document_management: FileDocumentManagement = test_container.applications.use_cases.managements.file_document()
text_document_management: TextDocumentManagement = test_container.applications.use_cases.managements.text_document()
web_document_management: WebDocumentManagement = test_container.applications.use_cases.managements.web_document()

long_form_qa_graph: LongFormQaGraph = test_container.applications.use_cases.graphs.long_form_qa()
passage_search_graph: PassageSearchGraph = test_container.applications.use_cases.graphs.passage_search()

process_passage_search: ProcessPassageSearch = test_container.applications.use_cases.passage_searches.process()
process_long_form_qa: ProcessLongFormQa = test_container.applications.use_cases.long_form_qas.process()

all_seeder: AllSeeder = test_container.seeders.all()

In [11]:
await all_seeder.up()

In [7]:
await all_seeder.down()

In [5]:
await two_datastore.async_client.set("test", "test", ex=10)

In [4]:
partition_document_processor: PartitionDocumentProcessor = PartitionDocumentProcessor(
    document_management=document_management,
    file_document_management=file_document_management,
    text_document_management=text_document_management,
    web_document_management=web_document_management,
)

summary_document_processor: SummaryDocumentProcessor = SummaryDocumentProcessor()
category_document_processor: CategoryDocumentProcessor = CategoryDocumentProcessor(
    summary_document_processor=summary_document_processor
)

In [12]:
state = State()
state.authorized_session = all_seeder.session_seeder.session_fake.data[0]
state.session = one_datastore.get_session()


In [13]:
from typing import List
from apps.inners.models.daos.document import Document
from sqlalchemy import text, ScalarResult

session = state.session
await session.close()
account_id: int = state.authorized_session.account_id
filter: dict = {
    "name": "NAME1",
}
size = 1000
filter_expressions: List[str] = []
for key, value in filter.items():
    # filter_expressions.append(f"account_document.{key}::text like '%{value}%'")
    filter_expressions.append(f"SIMILARITY({key}::text, '{value}')")
query: str = f"""
    SELECT *
    FROM document
    WHERE account_id = '{account_id}'
    ORDER BY (({'+'.join(filter_expressions)})/{len(filter_expressions)}) DESC
    LIMIT {size};
"""

found_document_result: ScalarResult[Document] = await session.exec(
    text(query)
)
found_documents: List[Document] = list(found_document_result.all())
found_documents

In [9]:
# await state.session.rollback()
# magic: Magic = Magic()
# magic.from_buffer(
#     all_seeder.file_document_seeder.file_document_fake.file_data[0]
# )
found_documents = await document_management.find_many_with_authorization_and_pagination(
    state=state,
    page_position=1,
    page_size=1000
)
found_file_documents = await file_document_repository.find_many_by_id_and_account_id(
    session=state.session,
    ids=[found_document.id for found_document in found_documents],
    account_id=state.authorized_session.account_id
)

In [12]:
found_documents
found_file_documents

In [6]:
passage_search_process_body = PassageSearchProcessBody(
    input_setting=PassageSearchInputSettingBody(
        document_ids=[
            all_seeder.document_seeder.document_fake.data[0].id,
            all_seeder.document_seeder.document_fake.data[1].id,
            all_seeder.document_seeder.document_fake.data[2].id
        ],
        llm_setting=LlmSetting(
            model_name="claude-3-haiku-20240307",
            max_token=500
        ),
        preprocessor_setting=PreprocessorSetting(
            is_force_refresh_categorized_element=False,
            is_force_refresh_categorized_document=False,
            chunk_size=500,
            overlap_size=50,
            is_include_table=False,
            is_include_image=False
        ),
        embedder_setting=EmbedderSetting(
            is_force_refresh_embedding=False,
            is_force_refresh_document=False,
            model_name="BAAI/bge-m3",
            query_instruction="Given the question, retrieve passage that answer the question."
        ),
        retriever_setting=RetrieverSetting(
            is_force_refresh_relevant_document=False,
            top_k=50
        ),
        reranker_setting=RerankerSetting(
            is_force_refresh_re_ranked_document=False,
            model_name="BAAI/bge-reranker-v2-m3",
            top_k=5
        ),
        question="what is political science?"
    )
)
passage_search_process_response = await process_passage_search.process(
    state=state,
    body=passage_search_process_body
)

In [7]:
passage_search_process_response

In [24]:
long_form_qa_process_body = LongFormQaProcessBody(
    input_setting=LongFormQaInputSettingBody(
        document_ids=[
            all_seeder.document_seeder.document_fake.data[0].id,
            all_seeder.document_seeder.document_fake.data[1].id,
            all_seeder.document_seeder.document_fake.data[2].id
        ],
        llm_setting=LlmSetting(
            model_name="claude-3-haiku-20240307",
            max_token=500
        ),
        preprocessor_setting=PreprocessorSetting(
            is_force_refresh_categorized_element=False,
            is_force_refresh_categorized_document=False,
            chunk_size=500,
            overlap_size=50,
            is_include_table=False,
            is_include_image=False
        ),
        embedder_setting=EmbedderSetting(
            is_force_refresh_embedding=False,
            is_force_refresh_document=False,
            model_name="BAAI/bge-m3",
            query_instruction="Given the question, retrieve passage that answer the question."
        ),
        retriever_setting=RetrieverSetting(
            is_force_refresh_relevant_document=False,
            top_k=50
        ),
        reranker_setting=RerankerSetting(
            is_force_refresh_re_ranked_document=False,
            model_name="BAAI/bge-reranker-v2-m3",
            top_k=5
        ),
        question="what is political science?",
        generator_setting=GeneratorSetting(
            is_force_refresh_generated_answer=False,
            is_force_refresh_generated_question=False,
            is_force_refresh_generated_hallucination_grade_hash=False,
            is_force_refresh_generated_answer_relevancy_grade_hash=False,
            prompt="""Instruction: Create a concise and informative answer for a given question based solely on the given passages. You must only use information from the given passages. Use an academic style. Do not repeat text. Cite at least one passage in each sentence. Cite the passages using passage number notation like "[number]". If multiple passages contain the answer, cite those passages like "[number, number, etc.]". If the passages do not contain the answer to the question, then say that answering is impossible given the available information with the explanation. Ensure the output is not re-explaining the instruction.
            Passages:
            {% for passage in passages %}
            [{{ loop.index }}]={{ passage.page_content }}
            {% endfor %}
            Question: {{ question }}
            Answer:"""
        ),
        transform_question_max_retry=3
    )
)
long_form_qa_process_response = await process_long_form_qa.process(
    state=state,
    body=long_form_qa_process_body
)

In [6]:
long_form_qa_process_response

In [8]:
converted_document_data = await libre_office_document_converter.convert(
    state=state,
    document_id=all_seeder.document_seeder.document_fake.data[1].id,
    output_format="pdf"
)
marked_document_data = await marker_document_converter.convert(
    input_file_data=converted_document_data,
    highlights=[("label", "Political science")]
)

In [6]:
elements = await partition_document_processor.partition(
    state=state,
    document_id=all_seeder.document_seeder.document_fake.data[0].id,
    file_partition_strategy="auto",
)

In [7]:
categorized_elements = await category_document_processor.categorize_elements(
    elements=elements
)
categorized_elements.texts = categorized_elements.texts[:]
categorized_documents = await category_document_processor.get_categorized_documents(
    categorized_elements=categorized_elements,
    summarization_model=ChatAnthropic(
        anthropic_api_key=one_llm_setting.LLM_ONE_ANTHROPIC_API_KEY_ONE,
        model="claude-3-haiku-20240307",
        max_tokens=100,
        streaming=True,
        temperature=0
    ),
    is_include_table=False,
    is_include_image=False,
    chunk_size=100,
    overlap_size=50,
)

In [37]:
from apps.inners.models.dtos.document_category import DocumentCategory
import uuid
from unstructured.chunking.basic import chunk_elements
from unstructured.documents.elements import Element
from typing import Dict, Any, Tuple, List
import more_itertools
from langchain_core.documents import Document

chunk_size: int = 400
overlap_size: int = 50
separators: Tuple[str, ...] = ("\n", " ")
id_key: str = "id"
document_category: DocumentCategory = DocumentCategory(
    texts=[],
    tables=[],
    images=[],
    id_key=id_key
)
chunked_texts: List[Element] = chunk_elements(
    elements=categorized_elements.texts,
    include_orig_elements=True,
    max_characters=chunk_size - overlap_size
)
for text_before, text_after in more_itertools.windowed(chunked_texts, n=2):
    orig_elements: List[Element] = []
    for orig_element in text_before.metadata.orig_elements + text_after.metadata.orig_elements:
        if not any(orig_element.id == existing_orig_element.id for existing_orig_element in orig_elements):
            orig_elements.append(orig_element)
    orig_elements_metadata: Dict[str, Any] = {
        "origin_metadata": [
            orig_element.metadata.to_dict() for orig_element in orig_elements
        ],
        "category": "text"
    }
    last_index_of_separators: int = -1
    for separator in separators:
        last_index_of_separator = text_before.text.rfind(separator, 0, len(text_before.text) - overlap_size)
        last_index_of_separators = max(last_index_of_separators, last_index_of_separator)

    text = text_before.text[last_index_of_separators + 1:] + " " + text_after.text
    document: Document = Document(
        page_content=text,
        metadata={
            id_key: str(uuid.uuid4()),
            **orig_elements_metadata,
        }
    )
    document_category.texts.append(document)


In [38]:
max_len = -1
for document in document_category.texts:
    print(document.page_content)
    print()
    max_len = max(max_len, len(document.page_content))

max_len
# for text in chunked_texts:
#     print(text.text)
#     print()
    
# len(chunked_texts)

In [ ]:
# cache_tool.clear_cache()
torch.cuda.empty_cache()
gc.collect()
cache_tool.get_cache()

In [7]:
input_state: PreparationGraphState = {
    "state": state,
    "document_ids": [
        all_seeder.document_seeder.document_fake.data[0].id,
        all_seeder.document_seeder.document_fake.data[1].id,
        all_seeder.document_seeder.document_fake.data[2].id
    ],
    "llm_setting": {
        "model_name": "claude-3-haiku-20240307",
        "max_token": 500,
        "model": None,
    },
    "preprocessor_setting": {
        "is_force_refresh_categorized_element": False,
        "is_force_refresh_categorized_document": False,
        "file_partition_strategy": "auto",
        "chunk_size": 500,
        "overlap_size": 50,
        "is_include_table": False,
        "is_include_image": False,
    },
    "categorized_element_hashes": None,
    "categorized_documents": None,
    "categorized_document_hashes": None,
}
preparation_graph: PreparationGraph = PreparationGraph(
    one_llm_setting=one_llm_setting,
    two_datastore=two_datastore,
    partition_document_processor=partition_document_processor,
    category_document_processor=category_document_processor
)
output_state: PreparationGraphState = await preparation_graph.compiled_graph.ainvoke(
    input=input_state
)

output_state

In [ ]:
model_name = "SeaLLMs/SeaLLM-7B-v2.5"
tokenizer = AutoTokenizer.from_pretrained(
    pretrained_model_name_or_path=model_name
)
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name,
)
pipe = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=8192,
    torch_dtype=torch.bfloat16,
    device_map="cuda"
)
llm = HuggingFacePipeline(pipeline=pipe)
chat_llm = ChatHuggingFace(
    llm=llm
)

generator_llm = chat_llm
critic_llm = chat_llm

# generator_llm = ChatAnthropic(
#     model="claude-3-haiku-20240307",
#     anthropic_api_key=one_llm_setting.LLM_ONE_ANTHROPIC_API_KEY_ONE
# )
# critic_llm = ChatAnthropic(
#     model="claude-3-opus-20240229",
#     anthropic_api_key=one_llm_setting.LLM_ONE_ANTHROPIC_API_KEY_ONE
# )
embeddings = BgeM3Embedding(
    use_fp16=False,
    normalize_embeddings=False,
    return_colbert_vecs=False,
)

In [ ]:
generator = TestsetGenerator.from_langchain(
    generator_llm=generator_llm,
    critic_llm=critic_llm,
    embeddings=embeddings
)

document_id = input_state["document_ids"][0]
document_category = output_state["categorized_documents"][document_id]
documents = document_category.get_all()
test_set = generator.generate_with_langchain_docs(
    documents=documents,
    test_size=1,
    distributions={
        evolutions.simple: 0.5,
        evolutions.reasoning: 0.25,
        evolutions.multi_context: 0.25
    }
)

In [27]:
eval_set = test_set.to_dataset()
eval_set.rename_column(
    original_column_name="answer",
    new_column_name="ground_truth"
)

In [166]:
for index, eval in enumerate(eval_set):
    state: State = State()
    state.authorized_session = all_seeder.session_seeder.session_fake.data[0]
    state.session = one_datastore.get_session()
    input_state: LongFormQaGraphState = {
        "state": state,
        "document_ids": [
            # all_seeder.document_seeder.document_fake.data[0].id,
            all_seeder.document_seeder.document_fake.data[1].id,
            all_seeder.document_seeder.document_fake.data[2].id
        ],
        "llm_setting": {
            "model_name": "claude-3-haiku-20240307",
            "max_token": 500,
            "model": None,
        },
        "preprocessor_setting": {
            "is_force_refresh_categorized_element": False,
            "is_force_refresh_categorized_document": False,
            "chunk_size": 500,
            "overlap_size:": 50,
            "is_include_table": False,
            "is_include_image": False,
        },
        "categorized_element_hashes": None,
        "categorized_documents": None,
        "categorized_document_hashes": None,
        "next_document_id": None,
        "embedder_setting": {
            "is_force_refresh_embedding": False,
            "is_force_refresh_document": False,
            # "model_name": "intfloat/multilingual-e5-large-instruct",
            "model_name": "BAAI/bge-m3",
            "query_instruction": "Given the question, retrieve passage that answer the question.",
        },
        "retriever_setting": {
            "is_force_refresh_relevant_document": False,
            "top_k": 50,
        },
        "reranker_setting": {
            "is_force_refresh_re_ranked_document": False,
            "model_name": "BAAI/bge-reranker-v2-m3",
            "top_k": 5,
        },
        "embedded_document_ids": None,
        "next_categorized_document": None,
        "relevant_documents": None,
        "relevant_document_hash": None,
        "re_ranked_documents": None,
        "re_ranked_document_hash": None,
        "question": "what is political science?",
        "generator_setting": {
            "is_force_refresh_generated_answer": False,
            "is_force_refresh_generated_question": False,
            "is_force_refresh_generated_hallucination_grade": False,
            "is_force_refresh_generated_answer_relevancy_grade": False,
            "prompt": """Instruction: Create a concise and informative answer for a given question based solely on the given passages. You must only use information from the given passages. Use an academic style. Do not repeat text. Cite at least one passage in each sentence. Cite the passages using passage number notation like "[number]". If multiple passages contain the answer, cite those passages like "[number, number, etc.]". If the passages do not contain the answer to the question, then say that answering is impossible given the available information with the explanation. Ensure the output is not re-explaining the instruction.
            Passages:
            {% for passage in passages %}
            [{{ loop.index }}]={{ passage.page_content }}
            {% endfor %}
            Question: {{ question }}
            Answer:"""
        },
        "transform_question_max_retry": 0,
        "generated_answer": None,
        "generated_answer_hash": None,
        "generated_question": None,
        "generated_question_hash": None,
        "generated_hallucination_grade": None,
        "generated_hallucination_grade_hash": None,
        "generated_answer_relevancy_grade": None,
        "generated_answer_relevancy_grade_hash": None,
    }
    output_state = await long_form_qa_graph.compiled_graph.ainvoke(input_state)

    eval_set[index]["contexts"] = [document.page_content for document in
                                   output_state["categorized_documents"][document_id].get_all()]
    eval_set[index]["answer"] = output_state["generated_answer"]

In [23]:
# loading the V2 dataset
amnesty_qa = load_dataset("explodinggradients/amnesty_qa", "english_v2", trust_remote_code=True)

In [24]:
eval_set_2 = amnesty_qa["eval"].select(range(1))
eval_set_2

In [25]:
result = evaluate(
    dataset=eval_set_2,
    llm=critic_llm,
    embeddings=embeddings,
    metrics=[
        metrics.faithfulness,
        metrics.answer_relevancy,
        metrics.context_recall,
        metrics.context_precision,
        #     metrics.answer_correctness,
        #     metrics.context_relevancy,
        #     metrics.context_entity_recall,
    ],
)

In [26]:
result